# Aanbod huurwoningen in Nederland

In [2]:
import pandas as pd

pararius = pd.read_csv("../data/pararius_listings.csv")
pararius.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2112 entries, 0 to 2111
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Link              2112 non-null   object
 1   Huurprijs         2110 non-null   object
 2   Locatie           2110 non-null   object
 3   m2                2112 non-null   object
 4   Huurovereenkomst  1064 non-null   object
 5   Beschrijving      2110 non-null   object
dtypes: object(6)
memory usage: 99.1+ KB


### Opschonen

Om de dataset te kunnen analyseren moeten eerst een aantal kolommen een ander datatype krijgen, worden opgeschoond en waar nodig worden bijgevuld:

1. Er zijn twee links die geen waarden hebben voor `Huurprijs`, `Locatie` en `Beschrijving`. Deze links werken niet meer dus we verwijderen deze rijen.
2. De `Huurprijs` kolom moet worden omgezet naar een `float`.
    - Sommige van deze waarden worden als volgt weergeven: '€ 1.075 - 1.775 per month'. In dat geval berekenen we het gemiddelde van de prijs.
    - Voor sommige huurwoningen moet je de prijs opvragen, de waarde is dan 'Price on request'. Voor dat geval voegen we een nieuwe kolom toe `Verzoek` met als waarde `True` of `False`.
3. De `m2` moet ook worden omgezet naar `float`.
    - Hier geldt dezelfde uitzondering als bij de huurprijs. We berekenen ook het gemiddelde aantal vierkante meter in dit geval.
4. De `Locatie` kolom bevat een postcode en de naam van de buurt. Dit gaan we opsplitsen in twee kolommen.
5. De URL in de `Link` kolom bevat de stad waarin de woning staat. Hiervoor maken we een nieuwe kolom `Stad`.

In [3]:
# Vind de lege rijen en verwijder ze
lege_rijen = pararius[pararius[['Huurprijs', 'Locatie', 'Beschrijving']].isnull().all(axis=1)]
pararius = pararius.drop(lege_rijen.index)

In [4]:
def schoon_huurprijs(prijs):
    """Functie om de 'Huurprijs' kolom op te schonen."""
    if isinstance(prijs, float) or prijs is None:
        return prijs
    
    if "Price on request" in str(prijs):
        return None # Lege waarde voor de huurprijs
    
    prijs = prijs.replace("€", "").replace("per month", "").replace(",", "").strip()

    # Check of de huurprijs twee prijzen kan zijn
    if "-" in prijs:
        laag, hoog = prijs.split("-")
        laag = float(laag.strip())
        hoog = float(hoog.strip())
        return (laag + hoog) / 2 # Bereken het gemiddelde van de twee prijzen
    else:
        return float(prijs)


def schoon_m2(waarde):
    """Functie om de 'm2' kolom op te schonen."""
    if pd.isnull(waarde):
        return None
    
    waarde = waarde.replace("m²", "").replace("㎡", "").strip()

    # Bereken het gemiddelde als er twee waarden voor 'm2' zijn
    if "-" in waarde:
        laag, hoog = waarde.split("-")
        laag = laag.strip()
        hoog = hoog.strip()
        return (float(laag) + float(hoog)) / 2
    else:
        return float(waarde)


def krijg_buurt(locatie):
    """Functie die de 'Locatie' kolom opsplits in de postcode en buurt."""
    if pd.isnull(locatie):
        return None
    
    if "(" in locatie and ")" in locatie:
        buurt = locatie.split("(")[-1].split(")")[0].strip()
        return buurt
    else:
        return None


def krijg_stad(link):
    """Functie die de stad uit de URL in de 'Link' kolom haalt."""
    if pd.isnull(link):
        return None
    
    # Splits de URL op en haal de naam van de stad op
    try:
        return link.split("/")[4] #
    except IndexError:
        return None


# Huurprijs opschonen
pararius["Verzoek"] = pararius["Huurprijs"].apply(lambda x: True if "Price on request" in str(x) else False)
pararius["Huurprijs"] = pararius["Huurprijs"].apply(schoon_huurprijs)

# Vierkante meter opschonen
pararius["m2"] = pararius["m2"].apply(schoon_m2)

# Locatie opschonen
pararius['Buurt'] = pararius['Locatie'].apply(krijg_buurt)
pararius['Locatie'] = pararius['Locatie'].apply(lambda x: x.split('(')[0].strip() if pd.notnull(x) else x)

# Voeg de 'Stad' kolom toe
pararius["Stad"] = pararius["Link"].apply(krijg_stad)

pararius.head()

,Link,Huurprijs,Locatie,m2,Huurovereenkomst,Beschrijving,Verzoek,Buurt,Stad
0,https://www.pararius.com/apartment-for-rent/am...,1900.0,1012 ES,65.0,Unlimited period,"Description\r\nGreat Location , 1 bedroom apt ...",False,Burgwallen-Oude Zijde,amsterdam
1,https://www.pararius.com/apartment-for-rent/zo...,2100.0,2718 SJ,106.0,Unlimited period,Description\r\nUnfurnished 4-room apartment lo...,False,Lansinghage c.a.,zoetermeer
2,https://www.pararius.com/apartment-for-rent/de...,800.0,2563 BH,35.0,Unlimited period,Description\r\nSuper nice apartment on Laan va...,False,Valkenboskwartier,den-haag
3,https://www.pararius.com/apartment-for-rent/ti...,1725.0,5038 BW,84.0,Temporary rental,Description\r\nCan be rented for a maximum of ...,False,Binnenstad Oost,tilburg
4,https://www.pararius.com/apartment-for-rent/os...,1010.0,5341 EB,56.0,Unlimited period,Description\r\nNice apartment with balcony for...,False,Centrum-Zuid,oss


## 1. Middenhuurwoningen

Op 25 juni 2024 heeft de Eerste Kamer ingestemd met de [Wet betaalbare huur](https://www.vbk.nl/legalupdate/wet-betaalbare-huur-middenhuur-aangenomen). Dit betekent o.a. dat de huurprijzen van woningen t/m 186 punten (middenhuurwoningen) begrensd worden. Het middenhuursegment gaat dus bestaan uit woningen met een huurprijs van meer dan € 879,66 per maand en maximaal € 1.165,81.

We gaan dus woningen filteren tussen deze prijsgrenzen. Hoeveel woningen uit de gehele dataset behoren tot dit middenhuursegment?


In [5]:
# Filter het middenhuursegment
middenhuur = pararius[(pararius['Huurprijs'] > 879.66) & (pararius['Huurprijs'] < 1165.81)]

rijen = len(pararius)
middenhuur_rijen = len(middenhuur)

percentage_middenhuur = (middenhuur_rijen / rijen) * 100

print(f"Percentage huurwoningen die tot het middenhuursegment horen: {percentage_middenhuur:.2f}%")

Percentage huurwoningen die tot het middenhuursegment horen: 9.10%


Wat is de gemiddelde prijs en aantal vierkante meter voor een middenhuurwoning?

In [6]:
# Bereken de gemiddelden
gem_huurprijs = middenhuur["Huurprijs"].mean()
gem_m2 = middenhuur["m2"].mean()

print(f"Gemiddelde huurprijs voor een middenhuurwoning: €{gem_huurprijs:.2f}")
print(f"Gemiddeld aantal vierkante meter voor een middenhuurwoning: {gem_m2:.2f} m²")

Gemiddelde huurprijs voor een middenhuurwoning: €1024.83
Gemiddeld aantal vierkante meter voor een middenhuurwoning: 57.96 m²


In welke steden staan de meeste middenhuurwoningen?

In [9]:
top_5_steden = middenhuur["Stad"].value_counts().head(5)

print(top_5_steden)

Stad
groningen     25
rotterdam     15
maastricht    11
eindhoven      9
nijmegen       8
Name: count, dtype: int64


## 2. Puntenstelsel 